In [4]:
from probDE.car import car_init
from probDE.cython.KalmanODE import KalmanODE
from probDE.utils import indep_init
import numpy as np
from inference import inference

In [ ]:
%load_ext cython

In [2]:
def fitz(X_t, t, theta):
    "FitzHugh-Nagumo ODE function."
    a, b, c = theta
    n_state = len(X_t)//2
    V, R = X_t[0], X_t[n_state]
    return np.array([c*(V - V*V*V/3 + R), -1/c*(V - a + b*R)])

In [ ]:
%%cython
# cython: boundscheck=False, wraparound=False, nonecheck=False, initializedcheck=False
import numpy as np
cimport numpy as np
cimport cython
import scipy as sp
import scipy.stats
from scipy.integrate import odeint

DTYPE = np.double
ctypedef np.double_t DTYPE_t

cdef class LoglikeFun:
    cdef public int n_state, tmin, tmax
    cdef public object fun, kode
    
    def __init__(self, n_state, tmin, tmax, fun, kode=None):
        self.n_state = n_state
        self.tmin = tmin
        self.tmax = tmax
        self.fun = fun
        self.kode = kode
    
    cpdef double loglike(self, double[::1] x, double[::1] mean, double var):
        return np.sum(sp.stats.norm.logpdf(x=x, loc=mean, scale=var))
    
    cpdef double ode_nlpost(self, np.ndarray[DTYPE_t, ndim=1] phi, np.ndarray[DTYPE_t, ndim=2] Y_t,
                          np.ndarray[DTYPE_t, ndim=1] x0, double step_size, np.ndarray[DTYPE_t, ndim=1] phi_mean,
                          double phi_sd, double gamma):
        theta = np.exp(phi)
        tseq = np.linspace(self.tmin, self.tmax, self.tmax-self.tmin+1)
        X_t = odeint(self.fun, x0, tseq, args=(theta,))[1:]
        lp = self.loglike(Y_t.flatten(), X_t.flatten(), gamma)
        lp += self.loglike(phi, phi_mean, phi_sd)
        return -lp
    
    cpdef kalman_nlpost(self, double[::1] phi, double[::1] Y_t,
                        double[::1] x0, double step_size, double[::1] phi_mean,
                        double phi_sd, double gamma):
        theta = np.exp(phi)
        first_obs = int(1/step_size)
        n_skip = int(1/step_size)
        X_t = self.kode.solve(x0, theta)
        X_t = X_t[first_obs::n_skip, [0, self.n_state]].flatten()
        lp = self.loglike(Y_t, X_t, gamma)
        lp += self.loglike(phi, phi_mean, phi_sd)
        return -lp

In [2]:
from scipy.integrate import odeint
import numpy as np
import scipy as sp
import scipy.stats
import numdifftools as nd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
class inference:
    def __init__(self, state_ind, tmin, tmax, fun, kode=None):
        self.state_ind = state_ind
        self.tmin = tmin
        self.tmax = tmax
        self.fun = fun
        self.kode = kode
    
    def loglike(self, x, mean, var):
        return np.sum(sp.stats.norm.logpdf(x=x, loc=mean, scale=var))
    
    def simulate(self, x0, theta_true, gamma):
        tseq = np.linspace(self.tmin, self.tmax, self.tmax-self.tmin+1)
        X_t = odeint(self.fun, x0, tseq, args=(theta_true,))[1:,]
        e_t = np.random.default_rng().normal(loc=0.0, scale=1, size=X_t.shape)
        Y_t = X_t + gamma*e_t
        return Y_t
    
    def ode_nlpost(self, phi, Y_t, x0, step_size, phi_mean, phi_sd, gamma):
        theta = np.exp(phi)
        tseq = np.linspace(self.tmin, self.tmax, self.tmax-self.tmin+1)
        X_t = odeint(self.fun, x0, tseq, args=(theta,))[1:]
        lp = self.loglike(Y_t, X_t, gamma)
        lp += self.loglike(phi, phi_mean, phi_sd)
        return -lp
    
    def kalman_nlpost(self, phi, Y_t, x0, step_size, phi_mean, phi_sd, gamma):
        "Compute the negative loglikihood of Y_t using the Euler method."
        theta = np.exp(phi)
        first_obs = int(1/step_size)
        n_skip = int(1/step_size)
        X_t = self.kode.solve(x0, theta)
        X_t = X_t[first_obs::n_skip, self.state_ind]
        lp = self.loglike(Y_t, X_t, gamma)
        lp += self.loglike(phi, phi_mean, phi_sd)
        return -lp
    
    def euler(self, x0, step_size, theta):
        "Evaluate Euler approximation given theta"
        n_eval = int((self.tmax-self.tmin)/step_size)
        X_t = np.zeros((n_eval+1, len(x0)))
        X_t[0] = x0
        one_ind = int(1/step_size)
        n_skip = int(1/step_size)
        for i in range(n_eval):
            X_t[i+1] = X_t[i] + self.fun(X_t[i], step_size*i, theta)*step_size
        return X_t[one_ind::n_skip]
    
    def euler_nlpost(self, phi, Y_t, x0, step_size, phi_mean, phi_sd, gamma):
        "Compute the negative loglikihood of Y_t using the Euler method."
        theta = np.exp(phi)
        X_t = self.euler(x0, step_size, theta)
        lp = self.loglike(Y_t, X_t, gamma)
        lp += self.loglike(phi, phi_mean, phi_sd)
        return -lp
    
    def phi_fit(self, Y_t, x0, step_size, theta_true, phi_sd, gamma, kalman):
        "Compute the optimized the log theta and its variance given Y_t."
        phi_mean = np.log(theta_true)
        n_theta = len(theta_true)
        if kalman:
            obj_fun = self.kalman_nlpost
        else:
            obj_fun = self.euler_nlpost
        opt_res = sp.optimize.minimize(obj_fun, phi_mean+.1,
                                       args=(Y_t, x0, step_size, phi_mean, phi_sd, gamma),
                                       method='Nelder-Mead')
        phi_hat = opt_res.x
        hes = nd.Hessian(obj_fun)
        phi_fisher = hes(phi_hat, Y_t, x0, step_size, phi_mean, phi_sd, gamma)
        phi_cho, low = sp.linalg.cho_factor(phi_fisher)
        phi_var = sp.linalg.cho_solve((phi_cho, low), np.eye(n_theta))
        return phi_hat, phi_var
    
    def theta_sample(self, phi_hat, phi_var, n_samples):
        "Simulate theta given the log theta hat and its variance."
        phi = np.random.multivariate_normal(phi_hat, phi_var, n_samples)
        theta = np.exp(phi)
        return theta
    
    def theta_plot(self, theta_euler, theta_kalman, theta_true, step_sizes):
        "Plot the distribution of theta using the Kalman solver and the Euler approximation."
        n_size, _, n_theta = theta_euler.shape
        nrow = 2
        fig, axs = plt.subplots(nrow, n_theta, sharex='col', figsize=(20, 5))
        patches = [None]*(n_size+1)
        for col in range(n_theta):
            axs[0, col].set_title('$\\theta_{}$'.format(col))
            for row in range(nrow):
                axs[row, col].axvline(x=theta_true[col], linewidth=1, color='r', linestyle='dashed')
                axs[row, col].locator_params(axis='x', nbins=3)
                axs[row, col].set_yticks([])
            for i in range(n_size):
                if col==0:
                    patches[i] = mpatches.Patch(color='C{}'.format(i), label='h={}'.format(step_sizes[i]))
                sns.kdeplot(theta_euler[i, :, col], ax=axs[0, col])
                sns.kdeplot(theta_kalman[i, :, col], ax=axs[1, col])

        axs[0, 0].set_ylabel('Euler')
        axs[1, 0].set_ylabel('KalmanODE')
        patches[-1] = mlines.Line2D([], [], color='r', linestyle='dashed', linewidth=1, label='True $\\theta$')
        axs[0, -1].legend(handles=patches)
        fig.tight_layout()
        plt.show()
        return

In [ ]:
phi_mean = np.log(theta_true)
phi = phi_mean +.1
inf = inference(n_state1, tmin, tmax, fitz, kode)
%timeit inf.kalman_nlpost(phi, Y_t, x0, h, phi_mean, phi_sd, gamma)

In [ ]:
LF = LoglikeFun(n_state1, tmin, tmax, fitz, kode)
%timeit LF.kalman_nlpost(phi, Y_t.flatten(), x0, h, phi_mean, phi_sd, gamma)

In [ ]:
LF = LoglikeFun(n_state, tmin, tmax, fitz)
%timeit LF.ode_nlpost(phi, Y_t, x0, step_size, phi_mean, phi_sd, gamma)